In [3]:
import os 
import torch 
from torchvision.transforms import Resize, InterpolationMode
from torchvision.transforms.functional import hflip, vflip
from data import Test_Preprocessor
from configs.config import * 
from tqdm import tqdm
from PIL import Image
import glob
import numpy as np

In [4]:
class Evaluator:
    def __init__(self, model, image_transform, device='cuda'):
        self.model = model.to(device)
        self.image_transform = image_transform
        self.device = device
    
    @torch.no_grad()
    def _predict(self, path, mask_mode='color', do_tta=True, vote_mode='any'):         
        x = self.image_transform(Image.open(path).convert('RGB'), None)[0].to(self.device)
        if do_tta: 
            X, tta_fns = self.forward_TTA(x)
            mask = self.backward_TTA(model(X), tta_fns)
            if vote_mode == 'any': 
                mask = (mask > 0.5).any(dim=0).squeeze()
            elif vote_mode == 'soft': 
                mask = (mask.mean(dim=0).squeeze() > 0.5)
        else: 
            mask = model(x.unsqueeze(0)).squeeze() > 0.5

        if mask_mode ==  'color': 
            mask = torch.where(mask, 255, 0)
            self._check_range(mask, [0, 255])
        elif mask_mode == 'class': 
            mask = torch.where(mask, 1, 0)
            self._check_range(mask, [0, 1])
        return mask
    
    def evaluate(self, image_paths, label_paths, do_tta=True, vote_mode='any'): 
        assert len(image_paths) == len(label_paths)
        total_score = 0 
        self.model.eval()
        for image_path, label_path in tqdm(list(zip(image_paths, label_paths))): 
            gt = torch.from_numpy(np.load(label_path)['image']).to(self.device)
            mask = self._predict(image_path, mask_mode='class', do_tta=do_tta, vote_mode=vote_mode)
            mask = Resize(gt.shape, InterpolationMode.NEAREST)(mask.unsqueeze(0)).squeeze()
            total_score += Evaluator.dice_score(mask, gt)
        return total_score/len(image_paths)            
    
    def test(self, paths, save_dir, mask_mode='color', do_tta=True, vote_mode='any'): 
        assert mask_mode in ('color', 'class')
        assert vote_mode in ('any', 'soft')
        assert hasattr(paths, '__iter__'), "Input Variable paths is not iterable!"
        for path in paths: 
            assert os.path.isfile(path), f"'{path}' does not exist!"
        
        self.model.eval()
        for path in tqdm(paths): 
            mask = self._predict(path, mask_mode, do_tta, vote_mode)
            self._save(mask.cpu().numpy().astype(np.uint8), path, save_dir)
        
    def _save(self, mask, path, save_dir): 
        save_path = os.path.join(save_dir, os.path.basename(path).split(".")[0] + ".png")
        Image.fromarray(mask).save(save_path)
        
    def _check_range(self, mask, range_list): 
        res = torch.zeros_like(mask)
        for ele in range_list:
            res += (mask == ele).long()
        assert torch.all(res > 0)
        
    def forward_TTA(self, x):
        res = []
        tta_fns = [(x, y) for x in [lambda x: x, hflip] for y in [lambda x: x, vflip]]
        for aug_pair in tta_fns:
            res.append(aug_pair[1](aug_pair[0](x)))
        return torch.stack(res, dim=0), tta_fns 
    
    def backward_TTA(self, X, tta_fns): 
        res = []
        for index in range(len(tta_fns)):
            tta_fn = tta_fns[index]
            res.append(tta_fn[1](tta_fn[0](X.select(0, index))))
        return torch.stack(res, dim=0)

    @staticmethod
    def dice_score(pred, gt, ep=1e-8):
        return ((2 * pred * gt).sum()/(pred.sum() + gt.sum() + ep)).item()        

In [3]:
model = torch.load("./models/resunet.pt", map_location='cpu')
evaluator = Evaluator(model, Test_Preprocessor(test_img_size), 'cuda')

In [4]:
paths = glob.glob("./data/Public_Image/*.jpg")
save_dir = "./Public_Prediction/"
# evaluator.test(paths, save_dir, vote_mode='soft')

In [8]:
image_path_list = sorted(glob.glob(train_image_dir + "*" + img_suffix))
split_index = int(len(image_path_list) * train_ratio)
test_image_path_list = image_path_list[split_index:]
test_label_path_list = [os.path.join("./data/Annotations/", "label_" + os.path.basename(image_path).split(".")[0] + ".npz") for image_path in test_image_path_list]

print(len(test_image_path_list))

158


In [7]:
evaluator.evaluate(test_image_path_list, test_label_path_list, do_tta=False)

100%|██████████| 158/158 [00:15<00:00, 10.37it/s]


0.8468356507865689

In [8]:
print(evaluator.evaluate(test_image_path_list, test_label_path_list, do_tta=True, vote_mode='any'))
print(evaluator.evaluate(test_image_path_list, test_label_path_list, do_tta=True, vote_mode='soft'))

  1%|▏         | 2/158 [00:00<00:14, 10.65it/s]

0.833565526123194


100%|██████████| 158/158 [00:14<00:00, 10.64it/s]

0.8581861566422107


In [9]:
model = torch.load("./models/resunet.pt", map_location='cpu')
evaluator = Evaluator(model, Test_Preprocessor((480, 480)), 'cuda')

In [10]:
evaluator.evaluate(test_image_path_list, test_label_path_list, do_tta=False)

100%|██████████| 158/158 [00:14<00:00, 10.68it/s]


0.8379216458695599

In [11]:
print(evaluator.evaluate(test_image_path_list, test_label_path_list, do_tta=True, vote_mode='any'))
print(evaluator.evaluate(test_image_path_list, test_label_path_list, do_tta=True, vote_mode='soft'))

  1%|          | 1/158 [00:00<00:18,  8.67it/s]

0.826642942768109


100%|██████████| 158/158 [00:18<00:00,  8.74it/s]

0.850184780510166


In [12]:
!touch evaluate.py

In [13]:
!mkdir configs

In [14]:
import shutil

shutil.move("./config.py", "./configs/")

'./configs/config.py'